In [1]:
class Task:
    def __init__(self, title, description, priority, status, due_date, owner):
        self.title = title
        self.description = description
        self.priority = priority
        self.status = status
        self.due_date = due_date
        self.owner = owner

    def to_dict(self):
        return self.__dict__

In [2]:
import json

class TaskDatabaseHandler:
    def __init__(self, filename='Task_Manager.json'):
        self.filename = filename

    def load_tasks(self):
        try:
            with open(self.filename, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            return {}

    def save_tasks(self, tasks):
        with open(self.filename, 'w') as file:
            json.dump(tasks, file, indent=4)


In [3]:
from datetime import datetime,timedelta
class TaskManager:
    def __init__(self, owner):
        self.owner = owner
        self.db = TaskDatabaseHandler()
        self.tasks = self.db.load_tasks()

    def _generate_id(self):
        return str(len(self.tasks) + 1)

    def _get_owner_tasks(self):
        return {tid: t for tid, t in self.tasks.items() if t['owner'] == self.owner}

    def show_reminders(self):
        print("\n🔔 Task Reminders")
        now = datetime.now()
        upcoming = []
        overdue = []

        for task in self._get_owner_tasks().values():
                due_date = datetime.strptime(task['due_date'], "%Y-%m-%d")
                if due_date < now:
                    overdue.append(task)
                elif due_date <= now + timedelta(days=1):
                    upcoming.append(task)

        if overdue:
            print("\n⚠️ Overdue Tasks:")
            for t in overdue:
                print(f"- {t['title']} (Was due: {t['due_date']})")

        if upcoming:
            print("\n⏰ Upcoming Tasks (due within 24h):")
            for t in upcoming:
                print(f"- {t['title']} (Due: {t['due_date']})")

        if not overdue and not upcoming:
            print("✅ No upcoming or overdue tasks!")

    
    def create_task(self):
        print("\n📋 Create Task")

        owner_tasks = self._get_owner_tasks()
        while True:
            title = input("Enter task title: ").strip()
            if any(task['title'].lower() == title.lower() for task in owner_tasks.values()):
                print("❌ You already have a task with this title. Please choose a different title.")
            else:
                break

        description = input("Enter description: ")
        
        while True:
            priority = input("Enter priority (Low, Medium, High): ").capitalize()
            if priority in ["Low", "Medium", "High"]:
                break
            else:
                print("❌ Invalid priority! Please enter 'Low', 'Medium', or 'High'.")
        
        valid_statuses = {
            "to do": "To-Do",
            "in progress": "In Progress",
            "completed": "Completed"
            }
        while True:
            status = input("Enter status (To-Do, In Progress, Completed): ").strip().lower()
            if status_input in valid_statuses:
                status = valid_statuses[status_input]  
                break
            else:
                print("❌ Invalid status! Please enter 'To-Do', 'In Progress', or 'Completed'.")
        
        
        while True:
            due_date = input("Enter due date (YYYY-MM-DD): ")
            try:
                parsed_date = datetime.strptime(due_date, "%Y-%m-%d")
                if parsed_date < datetime.now():
                    print("❌ Due date cannot be in the past. Please enter a valid date.")
                else:
                    break
            except ValueError:
                print("❌ Invalid date format! Please enter the date in the format YYYY-MM-DD.")  

        task_id = self._generate_id()
        task = Task(title, description, priority, status, due_date, self.owner)
        self.tasks[task_id] = task.to_dict()
        self.db.save_tasks(self.tasks)
        print("✅ Task created!")

    def view_tasks(self):
        print("\n📋 Your Tasks")
        owner_tasks = self._get_owner_tasks()
        if owner_tasks:
            for tid, t in owner_tasks.items():
                print(f"{tid}: {t['title']} - {t['status']} - Due: {t['due_date']}")
        else:
            print("❌ No tasks found.")

    def edit_task(self):
        print("\n✏️ Edit Task")
        title_to_edit = input("Enter the task title to edit: ").strip()
        owner_tasks = self._get_owner_tasks()
        found_id = None
        for tid, t in owner_tasks.items():
            if t['title'].lower() == title_to_edit.lower():
                found_id = tid
                break

        if not found_id:
            print("❌ Task not found.")
            return

        
        task = self.tasks[found_id]

        updates = {
            'title': input(f"New title [{task['title']}]: ").strip() or task['title'],
            'description': input(f"New description [{task['description']}]: ").strip() or task['description'],
            'priority': input(f"New priority [{task['priority']}]: ").capitalize().strip() or task['priority'],
            'status': input(f"New status [{task['status']}]: ").capitalize().strip() or task['status'],
            'due_date': input(f"New due date [{task['due_date']}]: ").strip() or task['due_date']
        }

        task.update(updates)
        self.db.save_tasks(self.tasks)
        print("✅ Task updated!")

    def delete_task(self):
        print("\n🗑️ Delete Task")
        title_to_delete = input("Enter the task title to delete: ").strip()
        owner_tasks = self._get_owner_tasks()
        found_id = None
        for tid, t in owner_tasks.items():
            if t['title'].lower() == title_to_delete.lower():
                found_id = tid
                break

        if found_id:
            del self.tasks[found_id]
            self.db.save_tasks(self.tasks)
            print("✅ Task deleted!")
        else:
            print("❌ Task not found or does not belong to you.")

    def search_task(self):
        print("\n🔍 Search Task")
        title = input("Enter part of the title to search: ").strip()
        results = {
            tid: task for tid, task in self._get_owner_tasks().items()
            if title.lower() in task['title'].lower()
        }
        if results:
            for tid, t in results.items():
                print(f"{tid}: {t['title']} - {t['status']} - Due: {t['due_date']}")
        else:
            print("❌ No matches found.")

    def filter_tasks(self):
        print("\n🎯 Filter Tasks")
        priority = input("Priority (Low, Medium, High or leave blank): ").capitalize().strip()
        valid_statuses = {
            "to do": "To-Do",
            "in progress": "In Progress",
            "completed": "Completed"
            }

        while True:
            status_input = input("Enter status (To-Do, In Progress, Completed): ").strip().lower()
            if status_input == "":
                status = ""  
                break
            elif status_input in valid_statuses:
                status = valid_statuses[status_input]  # format properly
                break
            else:
                print("❌ Invalid status! Please enter 'To-Do', 'In Progress', or 'Completed'.")
        due_date = input("Due date (YYYY-MM-DD or leave blank): ").strip()

        results = {
            tid: task for tid, task in self._get_owner_tasks().items()
            if (not priority or task['priority'] == priority) and
               (not status or task['status'] == status) and
               (not due_date or task['due_date'] == due_date)
        }

        if results:
            for tid, t in results.items():
                print(f"{tid}: {t['title']} - {t['status']} - {t['priority']} - Due: {t['due_date']}")
        else:
            print("❌ No matching tasks found.")

In [4]:
class TaskManagementSystem:
    def __init__(self, logged_in_email):
        self.manager = TaskManager(logged_in_email) 
        self.manager.show_reminders()
        self.task_menu()
    
    def task_menu(self):
        
        while True:
            print("\n🗂️ Task Management Menu")
            print("1. Create Task")
            print("2. View Tasks")
            print("3. Edit Task")
            print("4. Delete Task")
            print("5. Search Tasks")
            print("6. Filter Tasks")
            print("7. Exit")
            choice = input("Choose an option: ").strip()

            if choice == '1':
                self.manager.create_task()
            elif choice == '2':
                self.manager.view_tasks()
            elif choice == '3':
                self.manager.edit_task()
            elif choice == '4':
                self.manager.delete_task()
            elif choice == '5':
                self.manager.search_task()
            elif choice == '6':
                self.manager.filter_tasks()
            elif choice == '7':
                print("👋 Exiting... Goodbye!")
                break
            else:
                print("❌ Invalid choice. Try again.")